In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

np.random.seed(1337)
tf.random.set_seed(1337)

# Data

In [2]:
# Load the dataframes
train_data = pd.read_csv("data/train.csv")
test_data=pd.read_csv("data/test.csv")

# Preview the dataframes
display(train_data.head(10))
display(test_data.head(10))

,Sequence,Active
0,DKWL,0
1,FCHN,0
2,KDQP,0
3,FNWI,0
4,NKRM,0
5,IWHA,0
6,GEFV,0
7,KARS,0
8,IMGW,0
9,KYRK,0


,Sequence
0,HWFK
1,MWPW
2,ALDV
3,NTLG
4,LHYY
5,AFGM
6,TPNY
7,EAKD
8,NDKE
9,GFIS


In [3]:
def create_char_df(df):
    """Create a dataframe that contains four columns (one for each char in 'Sequence')."""

    # Split sequences into characters
    first = []
    second = []
    third = []
    fourth = []

    for _, row in df.iterrows():
        ch_1, ch_2, ch_3, ch_4 = row["Sequence"]

        first.append(ch_1)
        second.append(ch_2)
        third.append(ch_3)
        fourth.append(ch_4)

    df_dict = {
        "first": first,
        "second": second,
        "third": third,
        "fourth": fourth
    }
    return pd.DataFrame(df_dict)

X_df = create_char_df(train_data)
X_test_df = create_char_df(test_data)

display(X_df.head(5))
display(X_test_df.head(5))

,first,second,third,fourth
0,D,K,W,L
1,F,C,H,N
2,K,D,Q,P
3,F,N,W,I
4,N,K,R,M


,first,second,third,fourth
0,H,W,F,K
1,M,W,P,W
2,A,L,D,V
3,N,T,L,G
4,L,H,Y,Y


## OneHot Encoding

In [4]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(X_df)

display(enc.categories_)

# Ensure the onehots are the same for each letter
assert((enc.categories_[0] == enc.categories_[1]).all() and (enc.categories_[1] == enc.categories_[2]).all() and (enc.categories_[2] == enc.categories_[3]).all())

[array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
        'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], dtype=object),
 array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
        'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], dtype=object),
 array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
        'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], dtype=object),
 array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
        'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], dtype=object)]

# Training

In [55]:
from sklearn.model_selection import train_test_split

X = enc.transform(X_df).toarray()
y = train_data["Active"].values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.02, random_state=1337, stratify=y)

# Create training np arrays
#X_train = enc.transform(X_df).toarray().reshape(-1, 4, 20)
#y_train = train_data["Active"].values

In [56]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [57]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, GRU, Bidirectional, Dropout
import keras

batch_size = 128

model = Sequential()
#model.add(Bidirectional(GRU(20, input_shape=(4, 20))))
model.add(Dense(200, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(70, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(30, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=[f1_m]
)

history = model.fit(X_train, y_train, epochs=64, batch_size=batch_size, validation_data=(X_val, y_val))
print(model.summary())

Epoch 1/64
858/858 [==============================] - 2s 3ms/step - loss: 0.0842 - f1_m: 0.3786 - val_loss: 0.0428 - val_f1_m: 0.6987
Epoch 2/64
858/858 [==============================] - 3s 3ms/step - loss: 0.0397 - f1_m: 0.7484 - val_loss: 0.0238 - val_f1_m: 0.8435
Epoch 3/64
858/858 [==============================] - 2s 3ms/step - loss: 0.0301 - f1_m: 0.8131 - val_loss: 0.0202 - val_f1_m: 0.8731
Epoch 4/64
858/858 [==============================] - 2s 2ms/step - loss: 0.0256 - f1_m: 0.8341 - val_loss: 0.0216 - val_f1_m: 0.8471
Epoch 5/64
858/858 [==============================] - 2s 2ms/step - loss: 0.0225 - f1_m: 0.8587 - val_loss: 0.0156 - val_f1_m: 0.9138
Epoch 6/64
858/858 [==============================] - 3s 3ms/step - loss: 0.0203 - f1_m: 0.8704 - val_loss: 0.0168 - val_f1_m: 0.9003
Epoch 7/64
858/858 [==============================] - 3s 3ms/step - loss: 0.0184 - f1_m: 0.8895 - val_loss: 0.0160 - val_f1_m: 0.8933
Epoch 8/64
858/858 [==============================] - 2s 3ms/s

Epoch 62/64
858/858 [==============================] - 2s 3ms/step - loss: 0.0041 - f1_m: 0.9703 - val_loss: 0.0236 - val_f1_m: 0.9043
Epoch 63/64
858/858 [==============================] - 2s 3ms/step - loss: 0.0045 - f1_m: 0.9657 - val_loss: 0.0211 - val_f1_m: 0.8988
Epoch 64/64
858/858 [==============================] - 2s 3ms/step - loss: 0.0041 - f1_m: 0.9713 - val_loss: 0.0206 - val_f1_m: 0.9178
Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_84 (Dense)             multiple                  16200     
_________________________________________________________________
dropout_45 (Dropout)         multiple                  0         
_________________________________________________________________
dense_85 (Dense)             multiple                  14070     
_________________________________________________________________
dropout_46 (Dropout)         multiple       

In [58]:
from sklearn.metrics import f1_score
y_pred = np.around(model.predict(X_val))
f1_score(y_val, y_pred)

0.9268292682926829

In [9]:
X_test = enc.transform(X_test_df).toarray()

In [10]:
y_test=np.around(model.predict(X_test))

In [11]:
np.savetxt("submission.txt", y_test, fmt="%d")

In [12]:
display(y_test)

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)